This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5beta9) and depends on Magma (tested using version
2.26-9).

In this notebook, we identify curves of genus 7 with a rational $g^2_6$ (which does not square to the canonical class) which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 30 minutes for completion.

In [1]:
load("preamble.sage")

Construct the sets of $\mathbb{F}_{2^i}$-rational points of $\mathbf{P}^2 \times \mathbf{P}^2$ for $i=1,2$.

In [2]:
F = GF(2)
P.<x0,x1,x2,y0,y1,y2> = PolynomialRing(F, 6)

In [3]:
S1 = [vector(v) for v in ProjectiveSpace(F, 2)]
for v in S1:
    v.set_immutable()
S = list(itertools.product(S1, S1))
len(S)

49

Construct the group $\mathrm{GL}(3,\mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$, presented as a subgroup of $\mathrm{GL}(6, \mathbb{F}_2)$.

In [4]:
l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in GL(3,F).gens()] +\
       [block_matrix(2,2,[identity_matrix(3),0,0,g.matrix()], subdivide=False) for g in GL(3,F).gens()]
G0 = GL(6,F).subgroup(l0)
G0.order()

28224

Use an orbit lookup tree to find $\mathrm{GL}(3,\mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $\mathbb{P}^2 \times \mathbb{P}^2$. We require that no three points have the same image in either factor.

In [5]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=3,ncols=3)
    g2 = g.submatrix(row=3,col=3)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [6]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(3),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(6, F).subgroup(l0)

In [7]:
def optimized_rep(g):
    return g.matrix()

In [8]:
def forbid(vecs, easy=False):
    if len(vecs) == 3:
        if len(set(x[0] for x in vecs)) == 1 or len(set(x[1] for x in vecs)) == 1:
            return True
    return False

In [9]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep,
           'forbid': forbid}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)

In [10]:
nodes = {6: green_nodes(tree, 6), 7: green_nodes(tree, 7)}
len(nodes[6]), len(nodes[7])

(455, 1530)

For each 2-dimensional space of $(1,1)$-hypersurfaces passing through a representative tuple, solve linear equations to find candidates for a third hypersurface such that the triple intersection is exactly the given set. We also impose the desired point count over $\mathbb{F}_4$.

In [11]:
monos11 = [prod(i) for i in itertools.product([x0,x1,x2],[y0,y1,y2])]
monos22 = [prod(i) for i in itertools.product([prod(j) for j in itertools.combinations_with_replacement([x0,x1,x2],2)], 
                                              [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2],2)])]

In [12]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos22)) for x in S}

In [13]:
def redundancy(gens, F=F, monos11=monos11, monos22=monos22):
    return [vector(F, ((v*m).coefficient(mu) for mu in monos22)) for v in gens for m in monos11]

In [14]:
def vecs_to_gens(vecs):
    return [sum(vecs[0][i]*monos11[i] for i in range(9)),
            sum(vecs[1][i]*monos11[i] for i in range(9)),
            sum(vecs[2][i]*monos22[i] for i in range(36))]

In [15]:
curves = defaultdict(list)
for s1 in [6, 7]:
    for vecs in nodes[s1]:
        V = Matrix([coords1[x] for x in vecs]).right_kernel()
        for V1 in V.subspaces(2):
            (v1, v2) = V1.basis()
            gens1 = sum(v1[i]*monos11[i] for i in range(9))
            gens2 = sum(v2[i]*monos11[i] for i in range(9))
            pts = [x for x in S if gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
            perp = Matrix([coords2[x] for x in pts])
            target = vector(F, (0 if x in vecs else 1 for x in pts))
            for w in solve_right_iterator(perp, target, redundancy, [gens1, gens2]):
                gens3 = sum(w[i]*monos22[i] for i in range(36))
                curves[(s1,)].append((gens1, gens2, gens3))

In [16]:
[(s, len(curves[s])) for s in curves]

[((6,), 126834), ((7,), 57636)]

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2$ using point counting and for $i=3,4$ using commutative algebra.

In [17]:
F4 = GF(4)
S14 = [vector(v) for v in ProjectiveSpace(F4, 2)]
for v in S14:
    v.set_immutable()
S4 = list(itertools.product(S14, S14))
len(S4)

441

In [18]:
tmp = [s for s in curves if len(s) == 1]
for (s1,) in tmp:
    gens1 = 0
    gens2 = 0
    for gens in curves[(s1,)]:
        if (gens1, gens2) != (gens[0], gens[1]):
            (gens1, gens2) = (gens[0], gens[1])
            pts2 = [x for x in S4 if gens1(*x[0], *x[1]) == 0 and gens2(*x[0], *x[1]) == 0]
        gens3 = gens[2]
        s2 = sum(1 for x in pts2 if gens3(*x[0], *x[1]) == 0)
        if (s1 == 6 and s2 == 18) or (s1 == 7 and s2 == 15):
            curves[(s1,s2)].append((gens1, gens2, gens3))
    del curves[(s1,)]

In [19]:
[(s, len(curves[s])) for s in curves if len(s) == 2]

[((6, 18), 1774), ((7, 15), 4719)]

In [20]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + tuple(y^(2^n) + y for y in P.gens()))
    return (J.vector_space_dimension() - 2^(3*n+1) + 1) // (2^n-1)^2

In [21]:
for n in range(3, 5):
    tmp = set(t[:n] for t in targets7)
    tmp2 = [s for s in curves if len(s) == n-1]    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves if len(s) == n])

[((6, 18, 12), 448), ((7, 15, 7), 356)]
[((6, 18, 12, 18), 1), ((7, 15, 7, 31), 135)]


Close out this case.

In [22]:
I1 = P.ideal([x0,x1,x2])
I2 = P.ideal([y0,y1,y2])
CR = magma.CoxRing(P, [I1, I2], [[1,1,1,0,0,0],[0,0,0,1,1,1]], [])
proj = CR.ToricVariety()

In [23]:
closeout(curves, X=proj, genus=7)

Number of curves found: 0
Number of isomorphism classes found: 0
No covers found in this case!
Total time: 24 minutes
